In [1]:
# Establish a Pymongo connection with the MongoDB server
from IPython.display import Image
def get_db(db_name):
    from pymongo import MongoClient
    client = MongoClient('localhost:27017')
    db = client[db_name]
    return db
db = get_db("poker")

In [2]:
db.hands.find_one()

{u'_id': u'holdem_199504_798908096',
 u'board': [u'3h', u'7s', u'Kd', u'2c', u'7c'],
 u'dealer': 5,
 u'game_type': u'holdem',
 u'hand_num': 11775,
 u'num_players': 3,
 u'players': [{u'action': 10,
   u'bankroll': 483,
   u'bet_actions': [{u'actions': [u'blind bet', u'call'],
     u'stage': u'pre-flop'},
    {u'actions': [u'check'], u'stage': u'flop'},
    {u'actions': [u'check'], u'stage': u'turn'},
    {u'actions': [u'check'], u'stage': u'river'}],
   u'player_cards': [u'Jd', u'6d'],
   u'position': 1,
   u'username': u'alfalfa',
   u'winnings': 0},
  {u'action': 0,
   u'bankroll': 1260,
   u'bet_actions': [{u'actions': [u'fold'], u'stage': u'pre-flop'},
    {u'actions': [u'no action'], u'stage': u'flop'},
    {u'actions': [u'no action'], u'stage': u'turn'},
    {u'actions': [u'no action'], u'stage': u'river'}],
   u'player_cards': [],
   u'position': 3,
   u'username': u'bluftru',
   u'winnings': 0},
  {u'action': 10,
   u'bankroll': 7464,
   u'bet_actions': [{u'actions': [u'blind be

In [3]:
ct = db.hands.count()
print ct

2676657


In [5]:
# Question: How many hands exist for different numbers of players?
pipeline = [
    { "$match" : { "board.3" : { "$exists" : True }}},
    { "$group" : { "_id": "$num_players",
                  "count": {"$sum" : 1}}},
    { "$sort" : { "_id" : 1 }}

]
agg = db.hands.aggregate(pipeline)
for a in agg:
    print "Number of players: " + "{:,}".format(a["_id"]), "Count: " + "{:,}".format(a["count"])

Number of players: 2 Count: 145,388
Number of players: 3 Count: 172,845
Number of players: 4 Count: 182,631
Number of players: 5 Count: 178,101
Number of players: 6 Count: 197,992
Number of players: 7 Count: 214,211
Number of players: 8 Count: 231,798
Number of players: 9 Count: 263,970
Number of players: 10 Count: 411,345
Number of players: 11 Count: 23,077
Number of players: 12 Count: 24,325


In [11]:
# Question: How many hands exist where we know at least one player's cards, grouped and sorted by number of players
import time
t_start = time.time()
pipeline = [
    { "$unwind" : "$players" },
    { "$match" : { "players.player_cards" : { "$ne" : [] }}},
    { "$group" : { "_id": "$_id",
                 "num_players": { "$max": "$num_players" }}},   
    { "$group" : { "_id": "$num_players",
                  "count": {"$sum" : 1}}},
    { "$sort" : { "_id" : 1 }}

]
option = {"allowDiskUse" : "true"}
agg = db.hands.aggregate(pipeline, allowDiskUse=True)
for a in agg:
    print "Number of players: " + "{:,}".format(a["_id"]), "Count: " + "{:,}".format(a["count"])
t_end = time.time()
print "Seconds to process: " + "{:,}".format(int(t_end - t_start))

Number of players: 2 Count: 84,000
Number of players: 3 Count: 102,938
Number of players: 4 Count: 112,802
Number of players: 5 Count: 113,921
Number of players: 6 Count: 131,092
Number of players: 7 Count: 146,775
Number of players: 8 Count: 163,564
Number of players: 9 Count: 191,645
Number of players: 10 Count: 308,023
Number of players: 11 Count: 17,207
Number of players: 12 Count: 18,520
Seconds to process: 308


In [15]:
# Question: Who are the top 25 most frequent winners?
t_start = time.time()
pipeline = [
    { "$unwind" : "$players" },
    { "$match" : { "players.winnings" : { "$gt" : 0 }}},
    { "$group" : { "_id": "$players.username",
                 "hands_won": { "$sum": 1 }}},
    { "$sort" : { "hands_won" : -1 }},
    { "$limit" : 25 }
]
winners = db.hands.aggregate(pipeline, allowDiskUse=True)
for w in winners:
    print "Username: " + w["_id"] + ", Hands won: " + "{:,}".format(w["hands_won"])
t_end = time.time()
print "Seconds to process: " + "{:,}".format(int(t_end - t_start))


Username: kfish, Hands won: 22,183
Username: kwAAkbot, Hands won: 13,586
Username: Funky, Hands won: 11,815
Username: r00lbot, Hands won: 11,617
Username: Muck, Hands won: 11,198
Username: lokibot, Hands won: 8,642
Username: Travis, Hands won: 7,945
Username: Quick, Hands won: 7,930
Username: going, Hands won: 7,867
Username: Sonar, Hands won: 7,608
Username: is314onu, Hands won: 7,444
Username: loki, Hands won: 6,805
Username: MetroPam, Hands won: 6,798
Username: RiverRatt, Hands won: 6,764
Username: alfalfa, Hands won: 6,715
Username: coming, Hands won: 6,654
Username: edge, Hands won: 6,625
Username: show, Hands won: 6,585
Username: Calvert, Hands won: 6,548
Username: lynn, Hands won: 6,533
Username: CityPam, Hands won: 6,432
Username: loki2c, Hands won: 6,319
Username: SheDevil, Hands won: 6,182
Username: Grizz, Hands won: 6,152
Username: sagerbot, Hands won: 6,084
Seconds to process: 290


In [14]:
# Question: What's the deal with this guy Chris "JESUS" Ferguson?
t_start = time.time()
pipeline = [
    { "$unwind" : "$players" },
    { "$match" : { "players.username" : "JESUS" }},
    { "$group" : { "_id": "$players.username",
                 "hands_played": { "$sum": 1 }}}
]
jesus = db.hands.aggregate(pipeline, allowDiskUse=True)
for j in jesus:
    print "Username: " + j["_id"] + ", Hands played: " + "{:,}".format(j["hands_played"])
t_end = time.time()
print "Seconds to process: " + "{:,}".format(int(t_end - t_start))

Username: JESUS, Hands played: 140
Seconds to process: 288


In [19]:
# Let's try the Jesus question another (faster?) way:
t_start = time.time()
query = { "players" : { "username" : "JESUS" } }
jesus = db.hands.find(query)
print jesus
i = 0
for j in jesus:
    i = i + 1
print "Number of hands JESUS played: " + "{:,}".format(i)
t_end = time.time()
print "Seconds to process: " + "{:,}".format(int(t_end - t_start))

Number of hands JESUS played: 0
Seconds to process: 83
